# Continue writing functions to find the closest point on a route to our actual data point

Started by Nathaniel on Saturday, June 15, 2019

In [1]:
%load_ext autoreload
%autoreload 2

!date
!whoami

import numpy as np
import pandas as pd

Sat Jun 15 21:52:16 PDT 2019
ndbs


## Import my closest point module and read in a GTFS `shapes.txt` file

In [3]:
import find_closest_route_point as f

In [4]:
!ls ../data/source/gtfs_20180815/

agency.txt          calendar.txt        fare_rules.txt      stop_times.txt
block.txt           calendar_dates.txt  routes.txt          stops.txt
block_trip.txt      fare_attributes.txt shapes.txt          trips.txt


In [5]:
shapes_df = pd.read_csv('../data/source/gtfs_20180815/shapes.txt')
shapes_df.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,10002011,47.612137,-122.281769,1,0.0
1,10002011,47.612144,-122.281784,2,5.8
2,10002011,47.612148,-122.281830,3,13.5
3,10002011,47.612141,-122.281853,4,22.0
4,10002011,47.612102,-122.281921,5,45.0


# Testing, testing...

In [7]:
def get_two_things(x):
    return x+1, x-1

a = [1,2,3]
b = [get_two_things(x) for x in a]
b

[(2, 0), (3, 1), (4, 2)]

In [8]:
c = 3
c = -c
c

-3